AE with dense layer

In [1]:
import numpy as np
import itertools
import os, sys
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
import gpustat
gpustat.print_gpustat()

mustafar  Wed Jul  3 16:23:37 2019
[0] GeForce GTX 1070 | 50'C,   0 % |     0 /  8114 MB |
[1] GeForce GTX 1070 | 46'C,   0 % |     0 /  8114 MB |


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# from progressBar import ProgressBar
# from utils import EarlyStopping

In [5]:
dnd = []

## Get the dataset

In [6]:
from torch.utils import data

In [7]:
class Dataset(data.Dataset):
    def __init__(self, names = 'SM', 
                 template='../data/20190702_20part_PtOrder_v1/{}.npy'):
        if names == 'SM':
            self.SM_names = ['Wlnu', 'qcd', 'ttbar', 'Zll']
            names = self.SM_names
        self.names = names
        
        self.feature_names = ['Pt', 'eta', 'phi', 'charge', 'pId'][:3]
        
        for i, n in enumerate(names):
            ins = np.load(template.format(n)).astype(np.float32)[:,:,:3].reshape((-1, 60))
            lbs = i*np.ones(ins.shape[0]).astype(np.int8)
            
            print(n, ':', str(ins.shape[0]))
            
            if i == 0:
                self.inputs = ins
                self.labels = lbs
            else:
                self.inputs = np.concatenate((self.inputs, ins))
                self.labels = np.concatenate((self.labels, lbs))
    
    def __len__(self):
        return self.inputs.shape[0]
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.inputs[idx]

In [8]:
dataset = {}
# Use to do validation split
# torch.utils.data.random_split(dataset, lengths)
dataset['train'] = Dataset(names=['Wlnu'])

Wlnu : 100000


## Define the model

In [ ]:
class AEDenseNet(nn.Module):
    def __init__(self, N_features, dim_hidden, dim_latent, verbose = False):
        super(AEDenseNet, self).__init__()
        self.verbose = verbose
        
        self.p = N_features
        self.Nk = dim_latent
        
        self.encoder_modules = nn.ModuleDict({
            'PhiE': self.build_dense(dim_in=self.p,
                                   dim_out=self.Nk,
                                   dim_hidden=dim_hidden)
        })
        
        
        self.decoder_modules = nn.ModuleDict({
            'PhiD': self.build_dense(dim_in=self.Nk,
                                   dim_out=self.p,
                                   dim_hidden=dim_hidden)
        })
        
        
        self.onGPU = False
          
    def build_dense(self, dim_in, dim_out, dim_hidden):
        net = nn.Sequential(
                            nn.Linear(dim_in, dim_hidden),
                            nn.ELU(),
                            nn.Linear(dim_hidden, dim_hidden),
                            nn.ELU(),
                            nn.Linear(dim_hidden, dim_out),
#                             nn.ELU(),
                          )
        return net
        
    def encode(self, x):
        z = self.encoder_modules['PhiE'](x)
        return z
    
    def decode(self, z):
        x = self.decoder_modules['PhiD'](z)
        return x
        
        # --- Formatting x ----
        # pt
        pt = 1 + F.elu(x[:,0])
        # eta
        eta = 5*F.tanh(x[:,1])
        # phi
        phi = F.hardtanh(x[:,2], min_val=-3.14159, max_val=3.14159)
        # charge
#         x[:,:,3] = F.hardtanh(x[:,:,3], min_val=-1, max_val=2)
        # pId
#         x[:,:,4] = F.hardtanh(x[:,:,4], min_val=0, max_val=6)
        return torch.cat((pt,eta, phi), 1)
        
    
    def forward(self, x):
        z = self.encode(x)
        x_prime = self.decode(z)
        return x_prime
    
    def initWeights(self):
        def weights_init(M):
            if hasattr(M, 'weight'):
                nn.init.xavier_normal_(M.weight.data)
        
        self.apply(weights_init)
    
    def useGPU(self, N_GPU=1):
        if torch.cuda.is_available():
            print('Current device: {} ({} available)'.format(torch.cuda.current_device(), 
                                                             torch.cuda.device_count()))
            self.N_GPU = N_GPU
            torch.cuda.empty_cache()
            self.cuda(N_GPU)
            gpustat.print_gpustat()
            
            self.onGPU = True
        else: 
            print('GPU not available')

In [ ]:
model = AEDenseNet(
                   N_features=dataset['train'].inputs.shape[1],
                   dim_hidden=20,
                   dim_latent=10
                  )

print(model)
trainablePars = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

model.initWeights()

In [11]:
model.useGPU(0)

Current device: 0 (2 available)
mustafar  Wed Jul  3 16:23:46 2019
[0] GeForce GTX 1070 | 51'C,   8 % |   447 /  8114 MB | ocerri(437M)
[1] GeForce GTX 1070 | 46'C,   0 % |    10 /  8114 MB |


## Loss

In [ ]:
criterion = nn.MSELoss()

## Training

In [ ]:
training_params = {'batch_size': 10000,
                   'shuffle': True,
                   'num_workers': 0
                  }

max_epochs = 100

In [ ]:
torch.autograd.set_detect_anomaly(True);

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 
                              mode='min',
                              factor=0.3,
                              patience=2,
                              verbose=1,
                              threshold=1e-4,
                              cooldown=2,
                              min_lr=1e-7
                             )

# Early stopping
earlyStopping = EarlyStopping(patient=5)

In [ ]:
optimizer.zero_grad()
data_train_iter = data.DataLoader(dataset['train'], **training_params)

train_loss = []
for epoch in range(1, max_epochs+1):
    batch_loss = []
    pb = ProgressBar(len(data_train_iter), percentPrecision=5, headLabel='Ep {} '.format(epoch))
    for local_x, _ in data_train_iter:
        if model.onGPU:
            local_x = local_x.cuda()
        
        x_prime = model(local_x)
        loss = criterion(local_x, x_prime)
        batch_loss.append(loss.item())
        
        tail_label = 'Loss: {:2.2e}'.format(loss.item())
        if len(train_loss) > 0:
            tail_label += ' ({:2.2e})'.format(loss.item() - train_loss[-1][-1])
        pb.show(len(batch_loss)-1, tail_label=tail_label)
        loss.backward()
        optimizer.step()
    
    aux_loss = 0
    N_batches = 0
    for local_x, _ in data_train_iter:
        loss = criterion(local_x, model(local_x))
        aux_loss += loss.item()
        N_batches += 1
    batch_loss.append(aux_loss/N_batches)
    
    printout = 'Loss: {:2.2e}'.format(batch_loss[-1])
    if len(train_loss) > 0:
        printout += ' - Delta: {:2.2e}'.format(batch_loss[-1] - train_loss[-1][-1])
    if len(train_loss) > 1:
        d2L_de2 = batch_loss[-1] - 2*train_loss[-1][-1] + train_loss[-2][-1]
        printout +=' - Trend: {:2.2e}'.format(d2L_de2)
    print(printout+'\n')
    
    train_loss.append(batch_loss)
    
    if not earlyStopping.check(batch_loss[-1]):
        break

    scheduler.step(batch_loss[-1])

In [ ]:
train_loss = np.array(train_loss)
matplotlib.rcParams.update({'font.size': 16})

f = plt.figure(figsize=(16,8))

train_loss_flat = train_loss.flatten()
x = np.arange(1, train_loss_flat.shape[0]+1) * float(train_loss.shape[0])/train_loss_flat.shape[0]
plt.plot(x, train_loss_flat, '-', alpha=0.2, color='darkorange')

x = np.arange(1, train_loss.shape[0]+1)
plt.plot(x, train_loss[:,-1], 'o--', label='Train', color='darkorange')

plt.xlabel('Epoch')
plt.grid()
plt.legend(loc='best')
plt.yscale('log')
# f.savefig('plots/TrainingHistory_VAE_all-in-one_v'+VAEversion+'.png')

# f = open('TrainingHistory_VAE_all-in-one_v'+VAEversion+'.pkl', 'w')
# pickle.dump(fit_report.history, f, pickle.HIGHEST_PROTOCOL)
# f.close()

# Convergence check

In [ ]:
x = dataset['train'].inputs
model.eval()
x_prime = model(torch.from_numpy(dataset['train'].inputs[:10000])).cpu().detach().numpy()
z = model.encode(torch.from_numpy(dataset['train'].inputs[:10000])).cpu().detach().numpy()

### Latent space

In [ ]:
for i in range(z.shape[1]):
    x_aux = z[:,i]
    f = plt.figure(i)
    r = (np.min(x_aux), np.max(x_aux))
    plt.hist(x_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.xlabel('$z_{{{}}}$'.format(i))
    dnd.append(f)

### Output features

In [ ]:
for i in range(x_prime.shape[1]):
    x_aux = x[:,i].flatten()
    xp_aux = x_prime[:,i].flatten()
    f = plt.figure(i)
    r = (min(np.min(x_aux), np.min(xp_aux)), max(np.max(x_aux), np.max(xp_aux)))
    plt.hist(x_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.hist(xp_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.xlabel('Part {:.0f} '.format(int(i/3)) + dataset['train'].feature_names[i%3])
    dnd.append(f)